In [1]:
import pymongo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.preprocessing as skl

client = pymongo.MongoClient()
db = client.VentDyssynchrony_db
ventSettings = db.VentSettings_collection
breathData = db.BreathData_collection
RT = db.RTData_collection
RN = db.RNData_collection
patient = db. PatientData_collection

%matplotlib inline

In [2]:
def find_norms(x):
    if x['analysis.ds'] == False and x['analysis.pl'] == False and \
        x['analysis.pvt'] == False and x['analysis.ie'] == False:
            return True
    else: return False
    
def find_true_ds(x, ds_type):
    if x['analysis.'+ ds_type] == True and x['norms'] == False:
        return 1
    elif x['analysis.'+ ds_type] == False and x['norms'] == False:
        return np.nan
    elif x['analysis.'+ ds_type] == False and x['norms'] == True:
        return 0

In [3]:
patient_df = pd.read_csv('C:\Research_data\Demographic Data v2.csv', engine = 'c',
                             usecols = ['Study ID', 'PBW'])
patient_df.set_index(['Study ID'], inplace = True)
patient_df.head(15)

,PBW
Study ID,
100,84.5
101,68.4
102,52.4
103,47.8
104,79.9
105,68.4
106,61.6
107,45.5
108,54.7


In [73]:
data = breathData.find({},
                           {'patientID': 1, 'start_time': 1, 'breath_number': 1, 'characteristics.max_vol': 1,
                            'analysis':1, '_id': 0})

In [74]:
df = pd.io.json.json_normalize(data)
df['start_time'] = pd.to_datetime(df['start_time'])
df['norms'] = df.apply(find_norms, axis=1)

for items in ['ds', 'pl', 'pvt', 'ie']:
    df['true_'+items] = df.apply(find_true_ds, ds_type=items, axis=1)
    df.drop(labels='analysis.'+items, axis=1, inplace=True)
    
df.drop(labels=['analysis.fl', 'analysis.pds'], axis=1, inplace=True)

In [75]:
df.columns

Index(['analysis.validation', 'breath_number', 'characteristics.max_vol',
       'patientID', 'start_time', 'norms', 'true_ds', 'true_pl', 'true_pvt',
       'true_ie'],
      dtype='object')

In [134]:
df['PBW'] = np.nan
df.patientID = df.patientID.str.strip('P')

for rows in patient_df.iterrows():
    PBW = rows[1].ix[0]
    patientID = rows[0]
    
    df.loc[df.patientID == str(patientID), 'PBW'] = PBW
    
df['VT_PBW'] = df['characteristics.max_vol'] / df.PBW

In [135]:
df[['VT_PBW', 'characteristics.max_vol']].describe()

,VT_PBW,characteristics.max_vol
count,1784994.000000,1817239.000000
mean,6.540054,419.044139
std,2.144918,153.008430
min,-7.945205,-1075.000000
25%,5.508475,355.000000
50%,6.378866,410.000000
75%,7.322176,460.000000
max,82.467532,5117.500000


In [136]:
df.to_csv('c:\Research_data\VT_analysis.csv')